In [67]:
import os
import math
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torch.utils.data as data
import torchvision
from torchvision import transforms

import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import style

style.use('fivethirtyeight')
sns.set(style='whitegrid',color_codes=True)

In [68]:
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

EPOCHS = 10
BATCH_SIZE = 30
LEARNING_RATE = 0.003
IMG_SIZE = 64
CONV_SIZE = math.floor((((IMG_SIZE-2)/2)-2)/2)

TRAIN_DATA_PATH = "C:/Users/Administrator/Desktop/programming/datasets/images/flowers_train_test/train"
TEST_DATA_PATH = "C:/Users/Administrator/Desktop/programming/datasets/images/flowers_train_test/test"

TRANSFORM_IMG = transforms.Compose([
    transforms.Resize(IMG_SIZE),
    transforms.CenterCrop(IMG_SIZE),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                          std=[0.229, 0.224, 0.225])])

train_data = torchvision.datasets.ImageFolder(root=TRAIN_DATA_PATH, transform=TRANSFORM_IMG)
train_data_loader = data.DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True,  num_workers=4)
test_data = torchvision.datasets.ImageFolder(root=TEST_DATA_PATH, transform=TRANSFORM_IMG)
test_data_loader = data.DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)

In [69]:
print(len(train_data))
print(len(test_data))
train_data.classes

3813
510


['daisy', 'dandelion', 'rose', 'sunflower', 'tulip']

In [72]:
class Flower_Net_1(nn.Module):
    def __init__(self):
        super(Flower_Net_1,self).__init__()
        self.layer1 = nn.Sequential(nn.Conv2d(3,8,kernel_size=3,padding=1),nn.AvgPool2d(kernel_size=3,padding=1))
        self.layer2 = nn.Sequential(nn.Conv2d(8,16,kernel_size=3,padding=1),nn.MaxPool2d(kernel_size=3,padding=1))
        self.layer3 = nn.Sequential(nn.Conv2d(16,8,kernel_size=3,padding=1),nn.MaxPool2d(kernel_size=3,padding=1))
        self.flatten = nn.Flatten()

    def forward(self,x):
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))
        x = F.relu(self.layer3(x))
        out = self.flatten(x)
        return out

class Flower_Net_2(nn.Module):
    def __init__(self):
        super(Flower_Net_2,self).__init__()
        self.layer1 = nn.Sequential(nn.Conv2d(3,16,kernel_size=3,padding=1),nn.AvgPool2d(kernel_size=3,padding=1))
        self.layer2 = nn.Sequential(nn.Conv2d(16,32,kernel_size=3,padding=1),nn.MaxPool2d(kernel_size=3,padding=1))
        self.layer3 = nn.Sequential(nn.Conv2d(32,8,kernel_size=3,padding=1),nn.MaxPool2d(kernel_size=3,padding=1))
        self.flatten = nn.Flatten()

    def forward(self,x):
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))
        x = F.relu(self.layer3(x))
        out = self.flatten(x)
        return out
    
class Flower_Net_3(nn.Module):
    def __init__(self):
        super(Flower_Net_3,self).__init__()
        self.layer1 = nn.Sequential(nn.Conv2d(3,32,kernel_size=3,padding=1),nn.AvgPool2d(kernel_size=3,padding=1))
        self.layer2 = nn.Sequential(nn.Conv2d(32,8,kernel_size=3,padding=1),nn.MaxPool2d(kernel_size=3,padding=1))
        self.layer3 = nn.Sequential(nn.Conv2d(8,8,kernel_size=3,padding=1),nn.MaxPool2d(kernel_size=3,padding=1))
        self.flatten = nn.Flatten()

    def forward(self,x):
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))
        x = F.relu(self.layer3(x))
        out = self.flatten(x)
        return out

class ensemble_Net(nn.Module):    
    def __init__(self):
        super(ensemble_Net,self).__init__()
        f1 = Flower_Net_1()
        f2 = Flower_Net_2()
        f3 = Flower_Net_3()
        self.e1 = f1
        self.e2 = f2
        self.e3 = f3
        self.avgpool = nn.AvgPool1d(kernel_size=1)
        self.fc1 = nn.Linear(216,30)
        self.fc2 = nn.Linear(30,5)
    
    def forward(self,x):
        o1 = self.e1(x)
    
        o2 = self.e2(x)
        o3 = self.e3(x)
        x = torch.cat((o1,o2,o3),dim=1)
        #print(x.size())
        x = self.fc1(x)
        out = self.fc2(x)
        
        return out
    
        
model = ensemble_Net()
model

ensemble_Net(
  (e1): Flower_Net_1(
    (layer1): Sequential(
      (0): Conv2d(3, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): AvgPool2d(kernel_size=3, stride=3, padding=1)
    )
    (layer2): Sequential(
      (0): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): MaxPool2d(kernel_size=3, stride=3, padding=1, dilation=1, ceil_mode=False)
    )
    (layer3): Sequential(
      (0): Conv2d(16, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): MaxPool2d(kernel_size=3, stride=3, padding=1, dilation=1, ceil_mode=False)
    )
    (flatten): Flatten()
  )
  (e2): Flower_Net_2(
    (layer1): Sequential(
      (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): AvgPool2d(kernel_size=3, stride=3, padding=1)
    )
    (layer2): Sequential(
      (0): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): MaxPool2d(kernel_size=3, stride=3, padding=1, dilation=1, ceil_mode=False)
    )
 

In [73]:
model.to(DEVICE)
CNNmodel = ConvolutionalNetwork()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(CNNmodel.parameters(), lr=LEARNING_RATE)

In [75]:
import torch.nn.functional as F 
n_epochs = 8

for epoch in range(n_epochs):
    train_loss = 0
    val_loss = 0
    acc = 0.0
    print("Training....")
    model.train()
    for batch_num,(batch,labels) in enumerate(train_data_loader):
        inp,target = batch.to(DEVICE),labels.to(DEVICE)

        optimizer.zero_grad()
        output = model.forward(inp)
        
        op = F.softmax(output,dim=1)
        
        final_op = torch.argmax(op,dim=1)
    
        acc += torch.sum(final_op==target).item()/len(target)
        loss = criterion(output,target)
        
        loss.backward()
        optimizer.step()
        
        train_loss+=(loss.item()/len(batch))
        if batch_num%50 ==0 and batch_num!=0:
            print("TARGET: ",target)
            print("OUTPUT: ",final_op)
            print("Accuracy after ",batch_num,"steps: ",acc/batch_num)
        
    
    acc = acc/len(train_data_loader)
    print("Epoch: ",epoch,"Loss: ",train_loss," Accuracy: ",acc)
    
    
    eval_acc = 0

    model.eval()
    print("Validating.....")
    for batch in test_data_loader:
        inp,target = batch[0].to(DEVICE),batch[1].to(DEVICE)
        op = F.softmax(model.forward(inp))
        final_op = torch.argmax(op,dim=1)
        
        eval_acc += np.sum(final_op.detach().cpu().numpy()==target.detach().cpu().numpy())/len(target)
        
    print("Validation accuracy: ",eval_acc/len(test_data_loader))

Training....
TARGET:  tensor([4, 4, 3, 3, 3, 4, 1, 4, 4, 1, 4, 0, 0, 3, 0, 4, 1, 4, 1, 1, 4, 2, 3, 0,
        4, 4, 2, 1, 0, 4])
OUTPUT:  tensor([4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4])
Accuracy after  50 steps:  0.22666666666666657
TARGET:  tensor([0, 4, 3, 0, 2, 4, 3, 4, 2, 1, 3, 2, 1, 1, 1, 0, 0, 4, 3, 0, 3, 0, 0, 1,
        2, 0, 2, 0, 4, 4])
OUTPUT:  tensor([4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4])
Accuracy after  100 steps:  0.22699999999999995
Epoch:  0 Loss:  7.258926200866699  Accuracy:  0.23203124999999997
Validating.....


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:43: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Validation accuracy:  0.2
Training....


KeyboardInterrupt: 

In [ ]:
# class ConvolutionalNetwork(nn.Module):
#     def __init__(self):
#         super().__init__()
#         self.conv1 = nn.Conv2d(3, 8, 3, 1)
#         self.conv2 = nn.Conv2d(8, 16, 3, 1)
#         self.fc1 = nn.Linear(CONV_SIZE**2*16, 256)
#         self.fc2 = nn.Linear(256, 64)
#         self.fc3 = nn.Linear(64, 5)

#     def forward(self, x):
#         x = F.relu(self.conv1(x))
#         x = F.max_pool2d(x, 2, 2)
#         x = F.relu(self.conv2(x))
#         x = F.max_pool2d(x, 2, 2)
#         x = x.view(-1, CONV_SIZE**2*16)
#         x = F.relu(self.fc1(x))
#         x = F.relu(self.fc2(x))
#         x = self.fc3(x)
#         x = F.log_softmax(x, dim=1)
#         return x

In [ ]:
# import time
# start_time = time.time()

# train_loss = []
# test_loss = []
# train_correct = []
# test_correct = []

# for i in range(EPOCHS):
#     train_count = 0
#     test_count = 0
    
#     for b, (X_train, y_train) in enumerate(train_data_loader):
        
#         b+=1
        
#         y_pred = CNNmodel(X_train)
#         loss = criterion(y_pred, y_train)
 
#         predicted = torch.max(y_pred.data, 1)[1]
#         batch_corr = (predicted == y_train).sum()
#         train_count += batch_corr
        
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()

#         if b%30 == 0:
#             print(f'epoch: {i}   batch: {int(b/30)}   loss: {loss.item():.6f}')

#     train_loss.append(loss)
#     train_correct.append(train_count)

#     with torch.no_grad():
#         for b, (X_test, y_test) in enumerate(test_data_loader):
            
#             y_val = CNNmodel(X_test)

#             predicted = torch.max(y_val.data, 1)[1] 
#             test_count += (predicted == y_test).sum()

#     loss = criterion(y_val, y_test)
#     test_loss.append(loss)
#     test_correct.append(test_count)

# print(f'\nTime: {time.time() - start_time:.0f} s')

In [ ]:
torch.save(CNNmodel.state_dict(), 'cnn_model.pt')

In [ ]:
plt.plot(train_loss, label='train loss')
plt.plot(test_loss, label='test loss')
plt.legend()

In [ ]:
plt.plot(train_correct, label='train correct')
plt.plot(test_correct, label='test correct')
plt.legend()